In [1]:
#
# Preparacion
#
import matplotlib  #  type: ignore
import matplotlib.pyplot as plt  #  type: ignore
import pandas as pd  #  type: ignore

In [ ]:
#
"""
CHAT:
Cargue los datos de las tablas "drivers.csv" y "timesheet.csv" to ChatGPT.
"""
drivers = pd.read_csv(
    "../files/input/drivers.csv",
    sep=",",
    thousands=None,
    decimal=".",
)
timesheet = pd.read_csv(
    "../files/input/timesheet.csv",
    sep=",",
    thousands=None,
    decimal=".",
)

print("Tabla drivers")
display(drivers.head())
print()
print("Tabla timesheet")
display(timesheet.head())


In [ ]:
#
"""
PROMPT:
Calcule el promedio de las columnas "hours-logged" y "miles-logged" en la tabla "timesheet", agrupando los resultados por cada conductor (driverId).
"""
mean_timesheet = timesheet.groupby("driverId").mean()
mean_timesheet.pop("week")
mean_timesheet.head()

In [ ]:

"""
PROMPT:
Cree una tabla llamada "timesheet_with_means" basada en la tabla "timesheet", agregando una columna con el promedio de "hours-logged" para cada conductor (driverId).
"""
mean_hours_logged_by_driver = timesheet.groupby("driverId")["hours-logged"].transform(
    "mean"
)
timesheet_with_means = timesheet.copy()
timesheet_with_means["mean_hours-logged"] = mean_hours_logged_by_driver
timesheet_with_means.head()

In [ ]:

"""
PROMPT:
Cree una tabla llamada "timesheet_below" a partir de "timesheet_with_means", filtrando los registros donde "hours-logged" sea menor que "mean_hours-logged".
"""
timesheet_below = timesheet_with_means[
    timesheet_with_means["hours-logged"] < timesheet_with_means["mean_hours-logged"]
]
display(timesheet_below.head(), timesheet_below.tail())

In [ ]:

"""
PROMPT:
Cree una tabla llamada "sum_timesheet" agrupando la tabla "timesheet" por driverId y calculando la suma de las columnas "hours-logged" y "miles-logged".
"""
sum_timesheet = timesheet.groupby("driverId").sum()
sum_timesheet.head(10)

In [ ]:

"""
PROMPT:
Crea una tabla llamada "min_max_timesheet" a partir de la tabla "timesheet", agrupando los datos por driverId y calculando el valor mínimo y máximo de la columna "hours-logged".
"""
timesheet.groupby("driverId")["hours-logged"].agg(["min", "max"])


In [ ]:
"""
PROMPT:
Crea una tabla llamada "summary" combinando las tablas "sum_timesheet" y "drivers" mediante la clave driverId. De la tabla "drivers", incluye únicamente las columnas "driverId" y "name".
"""

# Unión de las tablas usando join
summary = pd.merge(
    sum_timesheet,
    drivers[["driverId", "name"]],
    on="driverId",
)
summary

In [10]:
"""PROMPT
Guarda la tabla "summary" en un archivo CSV con el nombre "summary.csv" para descargar.
"""
# Almacenamiento de los resultados
import os

if not os.path.exists("../files/output"):
    os.makedirs("../files/output")

summary.to_csv(
    "../files/output/summary.csv",
    sep=",",
    header=True,
    index=False,
)




In [ ]:

"""
PROMPT:
Crea una tabla llamada "top10" con los 10 conductores que tienen la mayor cantidad de millas registradas, ordenando los datos en forma descendente por la columna "miles-logged".
"""
# Ordenamiento por la cantidad de millas registradas
top10 = summary.sort_values(by="miles-logged", ascending=False).head(10)
top10

In [ ]:
"""

PROMPT:
Crea un gráfico de barras horizontales utilizando los datos de la tabla "top10", donde el eje y represente los nombres de los conductores y el eje x la cantidad de millas registradas. Asegúrate de que los conductores con más millas estén en la parte superior del gráfico.
"""

import os

top10 = top10.set_index("name")
top10["miles-logged"].plot.barh(color="tab:orange", alpha=0.6)

plt.gca().invert_yaxis()

plt.gca().get_xaxis().set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ","))
)

plt.xticks(rotation=90)

plt.gca().spines["left"].set_color("lightgray")
plt.gca().spines["bottom"].set_color("gray")
plt.gca().spines["top"].set_visible(False)
plt.gca().spines["right"].set_visible(False)

if not os.path.exists("../files/plots"):
    os.makedirs("../files/plots")

plt.savefig("../files/plots/top10_drivers.png", bbox_inches="tight")
